<a href="https://colab.research.google.com/github/RoseJaisil/Segmentation-Algorithms/blob/main/Augmentations_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Augmentation examples

Install libraries

In [2]:
!pip install -q kaggle

In [8]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"cvformedicalimages","key":"2f64bcdeb3bc442fd96bc4af68e1c862"}'}

In [9]:
!cp kaggle.json ~/.kaggle/

In [10]:
!chmod 600 ~/.kaggle/kaggle.json

In [11]:
!kaggle datasets download -d nikhilpandey360/chest-xray-masks-and-labels

100% 9.58G/9.58G [02:01<00:00, 139MB/s]
100% 9.58G/9.58G [02:01<00:00, 84.8MB/s]


In [ ]:
!unzip chest-xray-masks-and-labels.zip -d chest-xray-masks-and-labels

Let's explore the dataset

In [14]:
image_path = os.listdir("/content/chest-xray-masks-and-labels/Lung Segmentation/CXR_png/")

In [15]:
len(image_path)

800

In [16]:
mask_path = os.listdir("/content/chest-xray-masks-and-labels/Lung Segmentation/masks/")

In [17]:
len(mask_path)

704

In [18]:
# Create a folder
# !mkdir train_images

In [57]:
# make sub_folders for the 20 images and their respective masks
# !mkdir augmented
# !mkdir augmented/aug_images
# !mkdir augmented/aug_masks

In [51]:
# move 20 images from the main dataset to the new folders
# img_path = "/content/chest-xray-masks-and-labels/Lung Segmentation/CXR_png/"
# new_img_path = "/content/train_images/images/"
# msk_path = "/content/chest-xray-masks-and-labels/Lung Segmentation/masks/"
# new_msk_path = "/content/train_images/masks/"
# image_ids = next(os.walk(img_path))[2]
# mask_ids = next(os.walk(msk_path))[2]
# n = 1 
# for i in range(25):
#   mask_file_name = image_ids[i].split('.png')[0]+'_mask.png'
#   if mask_file_name in mask_ids:
#     shutil.move(img_path+image_ids[i], new_img_path)
#     shutil.move(msk_path+mask_file_name, new_msk_path)
#   n = n+1
#   if n == 20:
#     break


In [ ]:
# !zip -r sam_aug_data.zip /content/train_images

In [92]:
# if you want to remove files from a folder this is the code
folder_name = '/content/augmented/aug_masks'
for f in os.listdir(folder_name):
  os.remove(os.path.join(folder_name, f))

Until here the data has been explored and prepared. Now let's do the main part

Import libraries

In [54]:
import numpy as np
import shutil
from matplotlib import pyplot as plt
from skimage.transform import AffineTransform, warp
from skimage import io, img_as_ubyte
import random
import os
from scipy.ndimage import rotate
import albumentations as A
from tqdm import tqdm

Let's start

In [93]:
number_of_images_to_be_generated = 100

img_path = "/content/train_images/images/"
msk_path = "/content/train_images/masks/"
aug_img_path = "/content/augmented/aug_images/"
aug_msk_path = "/content/augmented/aug_masks/"

images = []
masks = [] 

# Reading image
print("Reading the input images name and stacking them in a list...")

for img in next(os.walk(img_path))[2]:
  images.append(os.path.join(img_path, img))

print("Reading the mask images name and stacking them in a list...")

for msk in next(os.walk(msk_path))[2]:
  masks.append(os.path.join(msk_path, msk))

# Composing the augmentor
print("Composing the augmentor...")
print("Note: Keras augmentation changes the image properties")

augmentor = A.Compose([
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.HorizontalFlip(p=1),
    A.Transpose(p=1),
    A.GridDistortion(p=1)
])


Reading the input images name and stacking them in a list...
Reading the mask images name and stacking them in a list...
Composing the augmentor...
Note: Keras augmentation changes the image properties


In [ ]:
i = 1 # initiator for the while loop
while i<=number_of_images_to_be_generated:
  # select an image randomly
  num = random.randint(0, len(images)-1)
  image_name = images[num]
  mask_name = masks[num]

  image = io.imread(image_name)
  
  mask = io.imread(mask_name)
  print(mask_name)

  # Good thing is you can augment both the image and mask with the same properties
  augmented_image_n_mask = augmentor(image=image, mask=mask)
  augmented_image = augmented_image_n_mask['image']
  augmented_mask = augmented_image_n_mask['mask']

  new_img_path = "%s/augmented_image_%s.png" %(aug_img_path, i)
  new_msk_path = "%s/augmented_mask_%s.png" %(aug_msk_path, i)
  io.imsave(new_img_path, augmented_image)
  io.imsave(new_msk_path, augmented_mask)
  i = i+1



Reference: @author: Sreenivas Bhattiprolu
https://github.com/bnsreenu